Topic modeling with title only

In [1]:
import pandas as pd
import os

references = pd.read_csv(os.path.join('..','results','paper_refs.csv'))

references

,Author,Title
0,"Russell S, Norvig P",Artificial Intelligence: a modern approach
1,"Witten IH, Frank E, Hall MA, et al",Data Mining: practical Machine Learning tools ...
2,"Zaki MJ, Meira Jr, W",Data Mining and analysis: fundamental concepts...
3,"Passfield L, Hopker JG",A mine of information: can sports analytics pr...
4,"Rein R, Memmert D",Big data and tactical analysis in elite soccer...
...,...,...
98,"Dalton-Barron NE, McLaren SJ, Black CJ, et al",Identifying contextual influences on training ...
99,"McLaren SJ, Weston M, Smith A, et al",Variability of physical performance and player...
100,"Oliveira WK, Jesus K, Andrade AD, et al",Monitoring training load in beach volleyball p...
101,"Düking P, Achtzehn S, Holmberg HC, Sperlich B",Integrated framework of load monitoring by a c...


In [3]:
#Using the nltk package for topic modeling
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/matteo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
from nltk.corpus import stopwords
from collections import Counter
import string

#Function for cleaning and preprocessing the titles
def preprocess_titles(titles):
    prepositions = set([
        # ... (list all prepositions here) ...
    ])
    stop_words = set(stopwords.words('english')).union(prepositions)

    cleaned_titles = []
    for title in titles:
        words = title.lower().translate(str.maketrans('', '', string.punctuation)).split()
        words = [word for word in words if word not in stop_words]
        cleaned_titles.extend(words)

    return cleaned_titles

#Applying topic modeling and showing the 15 most common words and their repetition frequency
cleaned_titles = preprocess_titles(references['Title'])
word_counts = Counter(cleaned_titles)
most_common_words = word_counts.most_common(15)

print(most_common_words)

[('football', 17), ('neural', 17), ('data', 16), ('training', 16), ('based', 16), ('team', 16), ('performance', 15), ('basketball', 14), ('artificial', 13), ('sports', 13), ('analysis', 12), ('network', 11), ('using', 10), ('injury', 10), ('mining', 9)]


Topic modeling with title and abstract

In [6]:
ref_abs = pd.read_csv(os.path.join('..','results','paper_refs_abstracts.csv'))

ref_abs

,Author,Title,Abstract,Year
0,"Russell S, Norvig P",Artificial Intelligence: a modern approach,From the enraged robots in the 1920 play R.U.R...,2015.0
1,"Witten IH, Frank E, Hall MA, et al",Data Mining: practical Machine Learning tools ...,NaN,NaN
2,"Zaki MJ, Meira Jr, W",Data Mining and analysis: fundamental concepts...,NaN,NaN
3,"Passfield L, Hopker JG",A mine of information: can sports analytics pr...,This paper explores the notion that the availa...,2017.0
4,"Rein R, Memmert D",Big data and tactical analysis in elite soccer...,Until recently tactical analysis in elite socc...,2016.0
...,...,...,...,...
98,"Dalton-Barron NE, McLaren SJ, Black CJ, et al",Identifying contextual influences on training ...,"Dalton-Barron, NE, McLaren, SJ, Black, CJ, Gra...",2021.0
99,"McLaren SJ, Weston M, Smith A, et al",Variability of physical performance and player...,The aims of this study were to establish sourc...,2021.0
100,"Oliveira WK, Jesus K, Andrade AD, et al",Monitoring training load in beach volleyball p...,NaN,NaN
101,"Düking P, Achtzehn S, Holmberg HC, Sperlich B",Integrated framework of load monitoring by a c...,Athletes schedule their training and recovery ...,2018.0
